In [23]:
import fitz
import re
import operator
import sentence_transformers
import sys
import os

In [24]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('paraphrase-distilroberta-base-v1')

In [25]:
def remove_tags(text):
    text = re.sub('\n'," ",text)
    return re.sub('\xa0',"",text)


In [26]:
user_input = input("Enter the path of your file: ")

assert os.path.exists(user_input), "I did not find the file at, "+str(user_input)
doc = fitz.open(user_input)
print("Path found")

Enter the path of your file: C:\Users\4a\mechinelearing\intership\Sementic search\Tolerance.pdf
Path found


In [28]:
text = []
for i in range(doc.page_count):
    page1 = doc.loadPage(i)
    page1text = page1.getText("text")
    page1text = [i.strip() for i in remove_tags(page1text).split(". ")]
    print(page1text)
    for x in page1text:
        if len(x)>1:
            text.append(x)

['Name:       Usama-bin-Tahir  Reg no:               17-NTU-1586  Department:         BS-IT (7th semester)  Submit to:              Sir Afit jamal  Presentation Topic:     Benefits of Tolerance in Islam   Due Date:              31-jan-2021      Tolerance       Tolerance is an important concept that helps people to live together        peacefully', "Tolerance also means that you don't put your opinions      above those of others, even when you       are sure that you are right", 'Tolerant people show strength in that they         can deal with different       opinions and perspectives', 'What Tolerance Tends to?     A Preliminary Report in the Quran tolerance is viewed a necessary  prerequisite for coexistence', 'It assumes social, cultural and religious  differences that require toleration, even if these differences do not  meet with the approval of the parties involved', 'Tolerance without  greater acceptance indicates a sense of conditioned approval', 'However, the Quran views the to

In [29]:
len_corpus = len(text)
input_text = input("Enter your text data: ")
ab = input_text.split(". ")
len_input_text = len(ab) 

Enter your text data: This implies that differences must not only be  expected, but be tolerated [15]', 'They are viewed as a cause for  celebration, hence al-Hujurat verse 13 recognizes diversity and  difference in term of gender, skin, color, language and belief, as well  as of that rank, and it asserts that differences are not only inherent  and will always be there but they are a blessing from God as the  Quran asserts in Hud verse118', 'Islamic Tolerance:   Practices from Quranic Perspective the Quran recognizes various  religious communities and a strong foundation for Quranic tolerance  can be seen in the following verses', '


In [30]:
input_precentage = input("Enter your percentage threshold:")



Enter your percentage threshold:0.65


In [31]:
embeddings1 = model.encode(input_text, convert_to_tensor=True)

In [32]:
sim_dict = {}
for t in text:
    ind = text.index(t)
    sent2 = ""
    for i in range(len_input_text):
        pick = ind +i
        if pick >= len_corpus:
            pass
        else:
            get_text = text[pick]
            sent2 = sent2 + ". "+ get_text

    embeddings2 = model.encode(sent2, convert_to_tensor=True)
    cosine_scores = util.pytorch_cos_sim(embeddings1, embeddings2)
    sim_dict[ind] = cosine_scores

In [33]:
sorted_d = sorted(sim_dict.items(), key=operator.itemgetter(1),reverse=True)

In [34]:
sorted_d[6]

(12, tensor([[0.6249]]))

In [35]:
final_text = {}
for value in sorted_d:
    index = value[0]
    if float(value[1].numpy()[0][0]) > float(input_precentage):
        final_text[index] = float(value[1].numpy()[0][0]) 
    

In [40]:
from docx import Document
from docx.shared import Inches
document = Document()
document.add_heading('Report', 0)

In [37]:
for x in final_text:
    index = x
    out = text[index]
    for i in range(1,len_input_text):
        out = out + ". "+text[index+i]
        
    print(out)
    doc= document.add_paragraph(out)
    print("\n\n")

They are viewed as a cause for  celebration, hence al-Hujurat verse 13 recognizes diversity and  difference in term of gender, skin, color, language and belief, as well  as of that rank, and it asserts that differences are not only inherent  and will always be there but they are a blessing from God as the  Quran asserts in Hud verse118



This aspect is highlighted in the Quranic discourse on tolerance    Diversity in Islam:     Identity and Influence From acknowledging differences in promoting  Islamic ethos the Quran outlines a firm standard for all, Muslim and  non-Muslims alike, based on the universal values of justice and  equality



This implies that differences must not only be  expected, but be tolerated [15]



Islamic Tolerance:   Practices from Quranic Perspective the Quran recognizes various  religious communities and a strong foundation for Quranic tolerance  can be seen in the following verses





In [39]:
document.save("usama.docx")